# Advanced GIS Term Paper: Wild Boar Risk for Vineyards

## Part 1: Vineyard Suitability


In [1]:
import arcpy
from arcpy import env
# import spatial analyst package for radiation tools
from arcpy.sa import *

In [ ]:
# importing data manually

# Mapping extent
# Atkis Roads and Tracks
# DEM, DSM
# Corine Landcover Germany 2018 + manually digitized Abandoned Vineyards

In [ ]:
# DEM derived Suitability factors
# Slope, Elevation

# DSM derived Suitability factors
# Solar Radiation

# Atkis: Distance to Roads/Tracks (Accessability)

# Corine LC
# Distance to Water Bodies (disregarded)
# Areas with different use that can not easily be converted (settlements,...)

# RLP Weinbergbodenkarte
# Frost danger areas (disregarded)

In [6]:
# Environment settings
arcpy.ResetEnvironments()

In [3]:
# Slope
slope = Slope("DEM_5m_update.tif", output_measurement = "PERCENT_RISE")

In [34]:
# save slope
slope.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\slope")

### Slope
Wichtiger Aspekt der Hangneigung ist die erhöhte Sonneneinstrahlung. Diese Bedingung wird allerdings hier <br>
durch die Radiation angedeckt, sodass ein sehr hoher slope Wert negativ ist und deshalb abgelehnt wird <br>
Die Erosion und der Arbeitsaufwand sind dann zu hoch <br>
100% als Maximum gewählt, da eine Weinbaufläche im südlichen Teil eine große Fläche mit 90%-100% Steigung hat<br>
Höhere Werte als 100% gibt es kaum in Weinbauflächen im Untersuchungsgebiet und hier wird für die Arbeit angenommen,<br>
dass es aufgrund der erschwerten Arbeit und höheren Erosion ungeeignet ist für Weinbau

In [4]:
# Layer with Slope < 100%
Slope100 = slope < 100

In [6]:
Slope100.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Slope100")

#### Gradient Factor of slope
Slope as limiting factor >100% to steep <br>
Slope as factor for accessability <br>
Slope as factor for fast cold air flow <br>
Slope as factor for erosion <br>
<br>
Optimal slope is 5% to 60% <br>
Accessability/Machinary and Erosion gets worse with increasing slope <br>
The Advantage of a slope greater than 5% is that cold air can flow down. <br>

<br>
-Normalize data to go from 0 to 1 (1 being 100%) and invert <br>
-normalize to have 1 as the highest value <br>
-data above 100% will be cutoff by the unsuitable layer<br>


In [3]:
# Calculation of slope suitability layer
slope_suit = arcpy.Raster("slope") / 100

In [3]:
# condition so a slope of 10% is optimal
slope_suit2 = Con( slope_suit, 1 - slope_suit, slope_suit*9, "VALUE >= 0.1")

In [4]:
# stays optimal until 60%
# set values over 100% slope to NoData
slope_suit60 = Con( slope_suit, slope_suit, "", "VALUE <= 1")

In [6]:
slope_suit60_2 = Con( ((slope_suit60 >= 0.1) & (slope_suit60 <= 0.6)) , 1, slope_suit60)

In [7]:
slope_suit60_3 = Con( (slope_suit60 < 0.1), 0, slope_suit60 )

In [8]:
slope_suit60_4 = Con( (slope_suit60 > 0.6), 2, slope_suit60_3 )

In [9]:
slope_suit60_5 = Con( ((slope_suit60 >= 0.1) & (slope_suit60 <= 0.6)), 1, slope_suit60_4 )

In [10]:
slope_suit60_6 = Con( slope_suit60_5 == 0, slope_suit60*10, slope_suit60_5 )

In [11]:
slope_suit60_7 = Con( slope_suit60_5 == 2, (1-slope_suit60)*2.5, slope_suit60_6 )

In [4]:
# normalize so highest value is 1
slope_suit3 = slope_suit2*(1+1/9)

In [12]:
slope_suit60_7.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\slope_suit60_7")

### Elevation 
Condition: not over 350m bc above is too cold <br>
Elevation will only be used as limiting factor and not weighted

In [28]:
# access DEM Layer
DEM = arcpy.Raster("DEM_5m_update.tif")
print(DEM)

DEM_5m_update.tif


In [29]:
# Über 350m ist es zu kalt
# create condition of Layer with DEM <= 350
DEM350 = DEM <= 350

In [36]:
DEM350.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DEM350")

### Radiation
Problems with cut-off number of radiation <br>
No fitting numbers for a minimum requirement of solar radiation was found in literature <br>
Even if i would have found a number there are several problems: <br>
Depending on type of vinegrape and climate the minimum is different <br>
I calculated the solar radiation with a model with parameters which were kept as default but can have a big effect on the overall number. One example is the Diffuse proportion which accounts for the diffusion effect in the atmosphere. <br>
Even if i would find a number, i won't know which model was used to create the number is or how acurate my model is. <br>
So as a solution I created the model and used the existing vineyards in the AOI to determine a minimum requirement of solar radiation. 



In [12]:
# load DSM raster and save in gdb
DSM = arcpy.Raster("D:\Geoinformatik\Advanced GIS\Term Paper\datasets\DSM_5m_update.tif")
DSM.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DSM")

In [2]:
# Define AreaSolarRadiation Parameters
skysize = 200
timeConfig = TimeMultipleDays(2020, 92, 305)  # 1. April bis 31. Oktober
# day_interval = default of 14 days (for calculation of sunmap)
hourinterv = 2   # calculation for every 2h of each day
# z_factor =  kept as default of 1
calcDirec = 16
# all kept at default {zenith_divisions}, {azimuth_divisions}, {diffuse_model_type}, {diffuse_proportion}, {transmittivity}
outDirectRad = "D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DirectRad"
outDiffuseRad = "D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DiffuseRad"
outDirectDur = "D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DirectDuration"


In [3]:
# perform Area Solar Radiation function
# was done once with DSM and once with DEM
AreaSolRad_DEM = AreaSolarRadiation("DEM_5m_update.tif", sky_size = skysize, time_configuration = timeConfig, hour_interval = hourinterv, 
                   calculation_directions = calcDirec  # , out_direct_radiation_raster = outDirectRad, 
                   #out_diffuse_radiation_raster = outDiffuseRad,
                   #out_direct_duration_raster = outDirectDur
                               )

In [4]:
AreaSolRad_DEM.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\AreaSolRad_DEM")

In [3]:
#AreaSolRad = arcpy.Raster("AreaSolRad")

In [1]:
# as kWh and save
AreaSol_DSM_kWh = arcpy.Raster("AreaSol_DSM1") / 1000
AreaSol_DSM_kWh.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\AreaSol_DSM_kWh")

In [4]:
# layer with Radation of over 600 kWH/m2
Sol600 = (AreaSolRad / 1000) < 600

In [5]:
Sol600.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Sol600")

#### Radiation Suitability Raster

In [6]:
# Normalize 
SolRadDSM_suit = arcpy.Raster("AreaSolRad")/1000000

In [7]:
SolRadDSM_suit.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\SolRadDSM_suit")

In [ ]:
# Import Corine Landcover Map
# Multiple landcover types are not suitable for vineyards i.e. urban areas or water bodies

#### Distance to roads 
-tracks can be easily created and distance to tracks is disregarded <br>
-roads are expensive and high distance to a road can be a limiting factor

In [1]:
# distance to roads set to 1000m
# create buffer around roads
roads_buffer1000 = arcpy.Buffer_analysis("atkis_road_Clip1", "roads1000", "1000 Meters", "FULL", 
                      "ROUND", "ALL")

In [28]:
# euclidian distance tool to create raster with distance to roads with distance to nearest road for every pixel
DistToRoad_raster = EucDistance("atkis_road_Clip1", cell_size = 5)

In [29]:
DistToRoad_raster.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DistToRoad_raster")

In [17]:
# Normalize
DistToRoad_raster_norm = DistToRoad_raster / 1963.14

In [19]:
DistToRoad_raster_norm.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\DistToRoad_raster_norm")

#### Combined Suitability (slope, radiation, distance to roads)

In [20]:
# Weights: Slope 3, Radiation 10, DistanceToRoad 1
Suitability3 = 3*arcpy.Raster("slope_suit60_7") + 10*arcpy.Raster("SolRadDSM_suit") - 1*arcpy.Raster("DistToRoad_raster_norm")

In [21]:
# clipped to mapping extent and saved

#### One layer for suitable/ not suitable areas

In [22]:
# convert polygons of corine landcover areas where vineyards are not possible 
# to raster
corine_notSuitable = arcpy.PolygonToRaster_conversion("Weinbau_nicht_moglich", "Klassenname", "corine_notSuitable", cellsize = 5)

In [37]:
# convert roads buffer to raster
roads1000_raster = arcpy.PolygonToRaster_conversion("roads1000", "OBJECTID" ,"roads1000_raster")

In [30]:
suitable2 = (arcpy.Raster("Sol600") == 0) & (arcpy.Raster("Slope100") == 1) & (
    arcpy.Raster("DEM350") == 1) & IsNull("corine_notSuitable") & (DistToRoad_raster <= 1000)

In [31]:
suitable2.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\suitable2")

#### Combining Suitability Layer and not suitable areas

In [44]:
Suitability_combined = arcpy.Raster("Suitability3_Clip") + 20*arcpy.Raster("suitable2") 

In [45]:
Suitability_combined = Con( Suitability_combined, Suitability_combined - 20, -1, "VALUE >= 20")

In [46]:
Suitability_combined = Con( IsNull(Suitability_combined), -1, Suitability_combined)

In [47]:
Suitability_combined.save("D:\Geoinformatik\Advanced GIS\AdvGIS term paper\AdvGIS term paper.gdb\Suitability_combined")